In [2]:
import os
import numpy as np
import pandas as pd
import xarray as xr
import tempfile
import matplotlib.pyplot as plt
import cartopy.crs as ccrs
from utils.utils import pretty_lat, pretty_lon, ProgressStatus, plot_n_steps_of_area_from_nc_file
from tempfile import TemporaryDirectory
from station.data_submission import DataSubmission
from station.station import StationData

from train_station_twin.training_executor import TrainingExecutor
from infilling.evaluation_executor import EvaluationExecutor
from infilling.infilling_writer import InfillingWriter

from era5.era5_for_station import DownloadEra5ForStation, DownloadEra5ForStationGaps
from era5.era5_download_hook import Era5DownloadHook
import matplotlib
%matplotlib inline
import matplotlib.pyplot as plt
import logging
logging.basicConfig(level=logging.INFO)

In [5]:
era5_hook = Era5DownloadHook(lat=0, lon=0)
era5_hook.coordinate_limits = {
    "north": 14.16443,
    "south": 12.16443,
    "west": -60.543,
    "east": -58.543
}

date = pd.Timestamp("2020-06-23")
folder = "testing"
era5_target_file_path = f"{folder}/era5.nc"
era5_hook.download_hours_in_same_day(
    year=date.year,
    month=date.month,
    day=date.day,
    hours=[18],
    target_folder=folder
)


Using URL: https://cds-beta.climate.copernicus.eu/api


2024-09-19 18:41:39,659 INFO Request ID is 5f32cd8b-abe4-442b-93d6-ea153822648e
INFO:cads_api_client.processing:Request ID is 5f32cd8b-abe4-442b-93d6-ea153822648e
2024-09-19 18:41:39,729 INFO status has been updated to accepted
INFO:cads_api_client.processing:status has been updated to accepted
2024-09-19 18:47:57,479 INFO status has been updated to successful
INFO:cads_api_client.processing:status has been updated to successful
INFO:multiurl.base:Downloading https://object-store.os-api.cci2.ecmwf.int:443/cci2-prod-cache/40ba586915c204fb6eec46885e4ec0f7.grib


In [7]:
from era5.era5_from_grib_to_nc import Era5DataFromGribToNc


# conversion
Era5DataFromGribToNc(
    folder_path=folder,
    era5_target_file_path=era5_target_file_path
)


print(xr.open_dataset(era5_target_file_path))

# plot the data
plot_n_steps_of_area_from_nc_file(
    path=era5_target_file_path,
    n=3,
    vars="tp",
    title="ERA5 temperature",
    vmin=300.6,
    vmax=301.1
)

Found 2020_6_23.grib
Merging files...



cdo    mergetime (Abort): Outputfile testing/era5.nc already exists!


CalledProcessError: Command 'cdo mergetime /var/folders/lj/q1ndb2493275c2y0vzyplnk00000gn/T/tmpftgfivx2/*.nc testing/era5.nc' returned non-zero exit status 1.

In [4]:
download_hook = Era5DownloadHook(lat=50, lon=10)


Using URL: https://cds-beta.climate.copernicus.eu/api


In [10]:
import cdsapi

client = cdsapi.Client()

dataset = 'reanalysis-era5-pressure-levels'
request = {
    'product_type': ['reanalysis'],
    'variable': ['geopotential'],
    'year': ['2024'],
    'month': ['03'],
    'day': ['01'],
    'time': ['13:00'],
    'pressure_level': ['1000'],
    'data_format': 'grib',
}
target = 'download.grib'

client.retrieve(dataset, request, target)

2024-09-19 17:50:27,201 INFO Request ID is aa5fbe56-9b25-4e36-b0e1-688f1caa9325
INFO:cads_api_client.processing:Request ID is aa5fbe56-9b25-4e36-b0e1-688f1caa9325
2024-09-19 17:50:27,286 INFO status has been updated to accepted
INFO:cads_api_client.processing:status has been updated to accepted
2024-09-19 17:54:44,799 INFO status has been updated to successful
INFO:cads_api_client.processing:status has been updated to successful
INFO:multiurl.base:Downloading https://object-store.os-api.cci2.ecmwf.int:443/cci2-prod-cache/7bc8dae330b944be482ec2a2f7255f0.grib


'download.grib'